In [1]:
from bs4 import BeautifulSoup
import requests
import zipfile
import os
import pandas as pd
from tqdm import tqdm
from urllib.parse import urljoin
import shutil

SESSION = requests.Session()
SURVEYS_DATABASE_PAGE_URL = 'https://survey.stackoverflow.co/'
DATA_FOLDER_PATH = 'local/data'

In [ ]:

def get_surveydata_links():
    response = SESSION.get(SURVEYS_DATABASE_PAGE_URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    data_gps_tracks = soup.find_all('a', {'data-gps-track': True, 'data-year': True})
    data_gps_tracks_links = [(urljoin(response.url, a['href']), a['data-year']) for a in data_gps_tracks]
    return data_gps_tracks_links


def download_surveydata_files(url: str, year):
    
    response = SESSION.get(url)
    
    # Extract the file type from the url
    file_type = url.split('.')[-1]

    # Assert that the file type is zip
    assert file_type == 'zip', f'File type {file_type} is not supported'
    
    # Save the file
    file_name = f'{year}.{file_type}'
    with open(file_name, 'wb') as f:
        f.write(response.content)
    
    # Extract the zip file
    path_to_extract = os.path.join(DATA_FOLDER_PATH, year)
    with zipfile.ZipFile(f'{year}.{file_type}', 'r') as zip_ref:
        zip_ref.extractall(path_to_extract)
    
    # Remove the zip file
    os.remove(file_name)

    # Remove __MACOSX folder if it exists
    macosx_folder_path = os.path.join(path_to_extract, '__MACOSX')
    if os.path.exists(macosx_folder_path) and os.path.isdir(macosx_folder_path):
        shutil.rmtree(macosx_folder_path)


def download_all_survey_data():
    data_gps_tracks_links = get_surveydata_links()
    for link, year in tqdm(data_gps_tracks_links):
        download_surveydata_files(link, year)
        # 5s per file
        # Remove __MACOSX folder

download_all_survey_data()


In [2]:
def get_csv_files():
    csv_files : dict[int, list] = {}
    data_folder_path_levels = len(DATA_FOLDER_PATH.split(os.sep))
    for root, dirs, files in os.walk(DATA_FOLDER_PATH):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                folders = root.split(os.sep)
                year = int(folders[data_folder_path_levels])
                if year not in csv_files:
                    csv_files[year] = []
                # Skip __MACOSX
                if '__MACOSX' in file_path:
                    continue
                csv_files[year].append(file_path)

    for year in csv_files:
        if year >= 2017:
            assert len(csv_files[year]) == 2, f'Year {year} has {len(csv_files[year])} files'
            # Assert that the 2 csv files are survey_results_schema.csv and survey_results_public.csv
            for file in csv_files[year]:
                assert file.endswith('survey_results_schema.csv') or file.endswith('survey_results_public.csv'), f'File {file} is not supported'
        else:
            assert len(csv_files[year]) == 1, f'Year {year} has {len(csv_files[year])} files'

    return csv_files

csv_files = get_csv_files()

def get_2016_schema_table():

    def get_readme_2016_filepath():
        for root, dirs, files in os.walk(os.path.join(DATA_FOLDER_PATH, '2016')):
            if '__MACOSX' in root:
                continue
            for file in files:
                if file.endswith('.txt'):
                    readme_file_path = os.path.join(root, file)
                    return readme_file_path

    readme_file_path : str = get_readme_2016_filepath()


    with open(readme_file_path, 'r') as f:
        lines = f.readlines()
    # Get all lines from "Database schema:" to the end
    for i, line in enumerate(lines):
        if 'Database schema:' in line:
            schema_table_lines = [line for line in lines[i+1:] if line.strip() != '']
            break

    schema_table_header = schema_table_lines[0].split(' --- ')
    schema_table_header = [cell.strip().strip("'").strip('"') for cell in schema_table_header]
    schema_table_rows = [line.split(' --- ') for line in schema_table_lines[1:]]
    schema_table_rows = [[cell.strip().strip("'").strip('"') for cell in row] for row in schema_table_rows]
    schema_table_rows = [['' if cell == 'N/A' else cell for cell in row] for row in schema_table_rows]

    schema_table_rows = [['Respondent', '', ''],] + schema_table_rows

    schema_table_df = pd.DataFrame(schema_table_rows, columns=schema_table_header)

    return schema_table_df

def get_dataframes(csv_files):

    dataframes = {}

    for year, files in tqdm(csv_files.items()):
        dataframes[year] = {}
        
        if year <= 2015:
            # If year <= 2015, the csv file has two top rows as headers
            assert len(files) == 1
            file_path = files[0]
            try:
                df_schema = pd.read_csv(file_path, nrows=2, header=None)
            except UnicodeDecodeError:
                df_schema = pd.read_csv(file_path, nrows=2, encoding='latin1', header=None)
            try:
                df_content = pd.read_csv(file_path, skiprows=2, header=None)
            except UnicodeDecodeError:
                df_content = pd.read_csv(file_path, skiprows=2, encoding='latin1', header=None)

            df_schema = df_schema.transpose().reset_index(drop=False)
            if year < 2015:
                df_schema.rename(columns={0: 'Question', 1: 'Answer', 'index': 'Column'}, inplace=True)
            else:
                df_schema.rename(columns={0: 'Answer', 1: 'Question', 'index': 'Column'}, inplace=True)
            dataframes[year] = {
                'schema': df_schema,
                'content': df_content,
            }
        elif year == 2016:
            file_path = files[0]
            df_schema = get_2016_schema_table()
            df_content = pd.read_csv(file_path)
            assert df_content['Unnamed: 0'].is_unique
            assert 'Respondent' not in df_content.columns
            df_content.rename(columns={'Unnamed: 0': 'Respondent'}, inplace=True)
            dataframes[year] = {
                'schema': df_schema,
                'content': df_content,
            }
        else:
            for file_path in files:
                if 'survey_results_public' in file_path:
                    content_file_path = file_path
                else:
                    assert 'survey_results_schema' in file_path
                    schema_file_path = file_path
            
            df_schema = pd.read_csv(schema_file_path)
            df_content = pd.read_csv(content_file_path)

            dataframes[year] = {
                'schema': df_schema,
                'content': df_content,
            }

    return dataframes

# 2011 - 2014
# 2015
# 2016
# 2017 - 2020
# 2021 - 2024

dataframes = get_dataframes(csv_files)

 14%|█▍        | 2/14 [00:00<00:01, 11.70it/s]C:\Users\An\AppData\Local\Temp\ipykernel_11888\3053088495.py:80: DtypeWarning: Columns (62) have mixed types. Specify dtype option on import or set low_memory=False.
  df_content = pd.read_csv(file_path, skiprows=2, header=None)
 29%|██▊       | 4/14 [00:00<00:01,  6.17it/s]C:\Users\An\AppData\Local\Temp\ipykernel_11888\3053088495.py:80: DtypeWarning: Columns (5,108,121,196,197,198) have mixed types. Specify dtype option on import or set low_memory=False.
  df_content = pd.read_csv(file_path, skiprows=2, header=None)
 50%|█████     | 7/14 [00:04<00:06,  1.09it/s]C:\Users\An\AppData\Local\Temp\ipykernel_11888\3053088495.py:113: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype opti

In [ ]:
all(dataframes[2021]['schema'].columns == dataframes[2022]['schema'].columns)
all(dataframes[2023]['schema'].columns == dataframes[2024]['schema'].columns)

In [ ]:
dataframes[2023]['schema']['type'].unique()

In [ ]:
dataframes[2024]['schema'][dataframes[2024]['schema']['type'] == 'CS']

In [ ]:
dataframes[2021]['content'][[c for c in dataframes[2021]['content'].columns if 'Language' in c]]

In [4]:
def match_unmatched_qnames(year, df_schema, df_content):
    assert year >= 2021
    unmatched_qnames = sorted(set(df_schema['qname']) - set(df_content.columns))
    unmatched_columns = sorted(set(df_content.columns) - set(df_schema['qname']))
    i = 0
    j = 0
    min_j_assessed = 0
    i_isfound = False
    columns_qname = {col:  None for col in unmatched_columns}
    qname_columns = {qname: [] for qname in unmatched_qnames}
    columns_qname_found = []
    while (i < len(unmatched_qnames)) and (min_j_assessed < len(unmatched_columns)):
        if unmatched_columns[j].startswith(unmatched_qnames[i]):
            i_isfound = True
            qname_columns[unmatched_qnames[i]].append(unmatched_columns[j])
            columns_qname[unmatched_columns[j]] = unmatched_qnames[i]
            j += 1
            min_j_assessed = j
        else:
            if i_isfound:
                i += 1
                i_isfound = False
            else:
                j += 1
            if not i_isfound and j == len(unmatched_columns):
                j = min_j_assessed
                i += 1
    return qname_columns, columns_qname

for year in range(2021, 2025):
    qname_columns, columns_qname = match_unmatched_qnames(year, dataframes[year]['schema'], dataframes[year]['content'])
    # assert columns_qname_notfound == ['ConvertedCompYearly', 'ResponseId']

In [12]:
year = 2024
qname_columns, columns_qname = match_unmatched_qnames(year, dataframes[year]['schema'], dataframes[year]['content'])
unmatched_columns = [col for col in columns_qname if columns_qname[col] is None]
unmatched_qnames = [qname for qname in qname_columns if qname_columns[qname] == []]

(['ConvertedCompYearly', 'ResponseId'],
 ['Frequency', 'JobSatPoints', 'Knowledge', 'SOTeamsUsage'])

In [23]:
dataframes[year]['schema'][dataframes[year]['schema']['qname'] == 'JobSatPoints']

,qid,qname,question,force_resp,type,selector
62,QID337,JobSatPoints,Assign points to the following attributes of y...,False,CS,VRTL


In [22]:
dataframes[year]['content'][dataframes[year]['content']['Frequency_1'].notna()]['Frequency_1']

10       3-5 times a week
12       1-2 times a week
15       1-2 times a week
18                  Never
20                  Never
               ...       
65268    1-2 times a week
65270    1-2 times a week
65291    1-2 times a week
65351               Never
65412    1-2 times a week
Name: Frequency_1, Length: 28369, dtype: object

In [8]:
[col for col in columns_qname if columns_qname[col] is None]

['ConvertedCompYearly', 'ResponseId']

In [ ]:
set(dataframes[2021]['content'].columns) - set(dataframes[2021]['schema']['qname'])

In [ ]:
set(dataframes[2021]['schema']['qname']) - set(dataframes[2021]['content'].columns)

In [ ]:
dataframes[2020]['schema']


In [ ]:
set(dataframes[2024]['schema']['qname'].unique()) - set(dataframes[2024]['content'].columns)

In [ ]:
dataframes[2024]['schema']['type'].unique()

In [ ]:
set(dataframes[2024]['content'].columns) - set(dataframes[2024]['schema']['qname'].unique())